# Library

In [101]:
# Librerie standard
import os
import random
import time
import re
from pathlib import Path
from collections import defaultdict, Counter
from itertools import islice

# Librerie per il trattamento delle immagini
import cv2
import imageio.v3 as imageio
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from torchvision.transforms import functional as TF
import torchvision.transforms as transforms

# Librerie per il machine learning e deep learning
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as func
import torchvision.models as models
from sklearn.svm import SVC

# Librerie per la gestione dei dati
import pandas as pd
import json
import orjson
import shutil 

# Librerie per il parallelismo e il multiprocessing
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor

# Librerie per il progresso e il monitoraggio
from tqdm import tqdm

# Librerie per la gestione dei dataset
from torch.utils.data import Dataset, DataLoader

# Librerie per modelli e trasformazioni in PyTorch
from torchvision import transforms

from collections import Counter
from sklearn.model_selection import train_test_split
import warnings

from torchvision import models
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch import optim
import torch
import json
import random
from collections import defaultdict
import ast

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from collections import defaultdict

import json
import ast
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import os

# Path

In [102]:
#Output folders and file names
COCO_JSON_NM = 'COCO_annotations_new.json'
OUT_COCO_JSON_NM = 'mod_COCO_annotations_new.json'
OUT_IMAGE_FLDR_NM = 'images'
OUT_CFG_FLDR_NM = 'YOLO_cfg'
RANDOM_SEED = 2023

in_dataset_pth = Path('/kaggle/input/our-xview-dataset')
out_dataset_pth = Path('/kaggle/working/')
img_fldr = Path(f'/kaggle/input/our-xview-dataset/{OUT_IMAGE_FLDR_NM}')
cfg_fldr_pth = Path(f'/kaggle/input/our-xview-dataset/{OUT_CFG_FLDR_NM}')

coco_json_pth = in_dataset_pth / COCO_JSON_NM
new_coco_json_pth = out_dataset_pth / OUT_COCO_JSON_NM
train_txt_pth = cfg_fldr_pth / 'train.txt'
val_txt_pth = cfg_fldr_pth / 'val.txt'
test_txt_pth = cfg_fldr_pth / 'test.txt'

#DATASET
train_path = '/kaggle/working/train.json'
test_path = '/kaggle/working/test.json'
val_path = '/kaggle/working/val.json'

random.seed(RANDOM_SEED)

In [103]:
# Pulizia dell'output per cartelle specifiche
def clean_output(output_dir):
    if output_dir.exists() and output_dir.is_dir():
        for item in output_dir.iterdir():
            if item.is_dir():
                shutil.rmtree(item)  # Rimuove la sotto-cartella
            else:
                item.unlink()  # Rimuove il file
        print(f"Cartella {output_dir} pulita.")
    else:
        print(f"Cartella {output_dir} non trovata. Nessuna azione necessaria.")

# Pulisce la cartella di output prima di avviare il processo
clean_output(out_dataset_pth)

Cartella /kaggle/working pulita.


In [104]:
# Sopprime i warning specifici del modulo skimage
warnings.filterwarnings("ignore", 
    message="Applying `local_binary_pattern` to floating-point images may give unexpected results.*")

# Background

In [105]:
def process_coco_json(input_path, output_path):
    """
    Funzione che prende il JSON COCO da un file di input, aggiunge un'annotazione di background per le immagini senza bbox,
    aggiunge la categoria "background" in formato mapping id:name, e salva il JSON modificato nel file di output.
    """
    # Leggi il JSON dal file di input
    with open(input_path, 'r') as f:
        data = json.load(f)

    # Ottieni le categorie
    categories = data.get('categories', [])

    # Rimappa la categoria "Aircraft" da id 0 a id 11
    for category in categories:
        if "0" in category and category["0"] == "Aircraft":
            category["0"] = "background"
    
    categories.append({"11": "Aircraft"})

    # Aggiorna le categorie nel JSON
    data['categories'] = categories

    # Crea un set per verificare quali immagini hanno annotazioni
    annotated_images = {ann['image_id'] for ann in data.get('annotations', [])}

    # Lista di nuove annotazioni da aggiungere per le immagini senza bbox
    new_annotations = []

    for image in data.get('images', []):
        image_id = image['id']
        if image_id not in annotated_images:
            # Crea una nuova annotazione per immagini senza annotazioni
            new_annotation = {
                'id': len(data['annotations']) + len(new_annotations),
                'image_id': image_id,
                'category_id': 0,  # Categoria background con ID 0
                'area': float(image['width'] * image['height']),  # Area della bbox
                'bbox': str([0.0, 0.0, float(image['width']), float(image['height'])])  # Bounding box come array
            }
            new_annotations.append(new_annotation)

    # Aggiungi le nuove annotazioni al JSON originale
    data['annotations'].extend(new_annotations)

    # Scrivi il JSON modificato nel file di output
    with open(output_path, 'w') as f:
        json.dump(data, f, indent=4)

In [106]:
process_coco_json(coco_json_pth, new_coco_json_pth)

# Splitting

In [107]:
def split_stratified(json_file, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    # Carica il JSON
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Raggruppare le annotazioni per category_id
    category_images = defaultdict(list)
    
    # Aggiungi solo le annotazioni valide (dove il bbox è valido)
    valid_annotations = []
    for annotation in data['annotations']:
        # Converte la stringa del bbox in una lista
        bbox = ast.literal_eval(annotation['bbox'])  # Converti la stringa in lista
        x_min, y_min, width, height = bbox
        x_max = x_min + width
        y_max = y_min + height
        
        if x_min <= x_max and y_min <= y_max:  # Bounding box valido
            valid_annotations.append(annotation)
            category_id = annotation['category_id']
            image_id = annotation['image_id']
            category_images[category_id].append(image_id)
    
    # Genera gli split per ogni category_id
    train_images, val_images, test_images = set(), set(), set()
    for category_id, image_ids in category_images.items():
        # Mescola gli image_id
        random.shuffle(image_ids)
        
        # Calcola i limiti per train, validation, e test
        total = len(image_ids)
        train_end = int(total * train_ratio)
        val_end = int(total * (train_ratio + val_ratio))
        
        # Aggiungi agli split
        train_images.update(image_ids[:train_end])
        val_images.update(image_ids[train_end:val_end])
        test_images.update(image_ids[val_end:])
    
    # Filtra le immagini e annotazioni per ciascuno split
    def filter_data(split_images):
        # Filtra solo le immagini e annotazioni con image_id presente in split_images
        filtered_images = [image for image in data['images'] if image['id'] in split_images]
        filtered_annotations = [annotation for annotation in valid_annotations if annotation['image_id'] in split_images]
        return {'images': filtered_images, 'annotations': filtered_annotations, 'categories': data['categories']}
    
    # Crea i nuovi JSON per train, validation, e test
    train_data = filter_data(train_images)
    val_data = filter_data(val_images)
    test_data = filter_data(test_images)
    
    # Salva i file JSON
    with open('train.json', 'w') as f:
        json.dump(train_data, f, indent=4)
    
    with open('val.json', 'w') as f:
        json.dump(val_data, f, indent=4)
    
    with open('test.json', 'w') as f:
        json.dump(test_data, f, indent=4)

In [108]:
# Chiamata della funzione
split_stratified(new_coco_json_pth)

# DataLoader

In [109]:
class CustomDataset(Dataset):
    def __init__(self, coco_json_file, img_dir, aug=False):
        """
        Inizializza il dataset personalizzato.
        Args:
        - coco_json_file: Il file JSON contenente le annotazioni.
        - img_dir: La cartella delle immagini.
        - aug: Booleano per attivare o meno l'augmentazione.
        - save_filtered_json: Se True, salva un file JSON filtrato.
        - filtered_json_path: Percorso del file JSON filtrato, se `save_filtered_json` è True.
        """
        def generate_id(file_name):
            return file_name.replace('_', '').replace('.jpg', '').replace('img', '')
        
        # Carica il file JSON delle annotazioni
        with open(coco_json_file, 'r') as f:
            coco_data = json.load(f)
        
        # Crea una struttura per le annotazioni
        self.image_annotations = {}
        self.image_bboxes = {}
        
        # Estrai le classi (categorie) dal file JSON
        self.classes = {}
        for category in coco_data['categories']:
            # Associa l'ID della categoria al nome, usando la chiave numerica come ID
            for category_id, category_name in category.items():
                self.classes[category_id] = category_name  # Associa l'ID categoria al nome
        
        # Filtra le annotazioni con bounding box validi
        valid_annotations = []
        for annotation in coco_data['annotations']:
            # Converti la stringa del bbox in una lista (se è una stringa)
            bbox = annotation['bbox']
            if isinstance(bbox, str):
                bbox = json.loads(bbox)  # Converte la stringa in lista
            x_min, y_min, width, height = bbox
            x_max = x_min + width
            y_max = y_min + height
            
            if x_min < x_max and y_min < y_max:  # Bounding box valido
                valid_annotations.append(annotation)
        
        # Aggiungi la mappa di annotazioni valide
        for annotation in valid_annotations:
            image_id = annotation['image_id']
            category_id = annotation['category_id']
            bbox = annotation['bbox']  # Formato COCO [x_min, y_min, width, height]
            
            if image_id not in self.image_annotations:
                self.image_annotations[image_id] = []
                self.image_bboxes[image_id] = []
            
            self.image_annotations[image_id].append(category_id)
            self.image_bboxes[image_id].append(bbox)
        
        # Mappa per associare ID immagine a file_name
        self.image_info = {
            image['id']: image['file_name']
            for image in coco_data['images']
        }
        
        # Filtra le immagini che non sono presenti nel JSON
        self.img_dir = img_dir
        self.image_paths = [
            os.path.join(img_dir, image['file_name'])
            for image in coco_data['images']
            if image['id'] in self.image_info  # Prendi solo le immagini presenti nel JSON
        ]
        
        # Trasformazioni di base e di augmentation
        self.base_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),   
        ])
        
        self.aug_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),
        ]) 
        
        self.aug = aug
    
    def save_filtered_json(self, coco_data, valid_annotations, filtered_json_path):
        """
        Salva un file JSON con annotazioni filtrate.
        Args:
        - coco_data: Dati COCO originali.
        - valid_annotations: Annotazioni valide filtrate.
        - filtered_json_path: Percorso del file JSON filtrato.
        """
        filtered_data = {
            "images": coco_data["images"],
            "annotations": valid_annotations,
            "categories": coco_data["categories"]
        }
        with open(filtered_json_path, 'w') as f:
            json.dump(filtered_data, f, indent=4)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        # Estrai il nome dell'immagine e l'ID corrispondente
        img_path = self.image_paths[index]
        img_name = os.path.basename(img_path)
        img_id = int(img_name.replace('_', '').replace('.jpg', '').replace('img', ''))
        
        if img_id not in self.image_info:
            raise ValueError(f"Immagine {img_name} non trovata nel file COCO")
        
        if not os.path.exists(img_path):
            raise ValueError(f"Immagine non trovata nel percorso: {img_path}")
        
        # Carica l'immagine
        image = Image.open(img_path).convert('RGB')
        original_width, original_height = image.size
        
        # Applica le trasformazioni
        if self.aug:
            image_tensor = self.aug_transform(image)
        else:
            image_tensor = self.base_transform(image)
        
        # Estrai le annotazioni e i bounding boxes
        categories = self.image_annotations.get(img_id, [])
        bboxes = self.image_bboxes.get(img_id, [])
        
        if not bboxes:  # Immagini senza annotazioni
            target = {
                "boxes": torch.zeros((0, 4), dtype=torch.float32),
                "labels": torch.zeros((0,), dtype=torch.int64)
            }
        else:
            # Converte da formato COCO [x_min, y_min, width, height] a [x_min, y_min, x_max, y_max]
            scale_x = 320 / original_width
            scale_y = 320 / original_height
            
            # Scaling dei bounding boxes
            scaled_bboxes = []
            for bbox in bboxes:
                if isinstance(bbox, str):
                    bbox = json.loads(bbox)  # Converte la stringa in lista se necessario
                x_min, y_min, width, height = bbox
                x_max = x_min + width
                y_max = y_min + height
                
                scaled_bboxes.append(torch.tensor([  
                    float(x_min) * scale_x,               # x_min
                    float(y_min) * scale_y,               # y_min
                    float(x_max) * scale_x,               # x_max
                    float(y_max) * scale_y                # y_max
                ], dtype=torch.float32))
            
            target = {
                "boxes": torch.stack(scaled_bboxes),
                "labels": torch.tensor(categories, dtype=torch.int64)
            }
        
        return image_tensor, target

In [110]:
def collate_fn(batch):
    """
    Funzione di collation per il DataLoader, utile per il batching di immagini e annotazioni.
    La funzione restituirà un batch di immagini e un batch di target, formattato correttamente per Faster R-CNN.
    
    Args:
    - batch: lista di tuple (image, target)
    
    Returns:
    - images: batch di immagini
    - targets: lista di dizionari contenenti le annotazioni per ogni immagine
    """
    # Separa immagini e target
    images, targets = zip(*batch)

    # Converte la lista di immagini in un batch di immagini
    images = list(images)

    # Restituisci il batch
    return images, list(targets)

In [111]:
# Creazione dei dataset
train_dataset = CustomDataset(train_path, img_fldr,  aug=True)
valid_dataset = CustomDataset(val_path, img_fldr, aug=False)  
test_dataset = CustomDataset(test_path, img_fldr, aug=False)  

# Creazione dei DataLoader
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

In [121]:
# Funzione per verificare i target con None
def check_none_target(data_loader):
    none_count = 0
    
    for images, targets in data_loader:
        for target in targets:
            # Controlla se il target è vuoto
            if target == None:  # Se il numero di box è 0, significa che non ci sono annotazioni
                none_count += 1
    
    return none_count

# Controlla i target nei DataLoader
train_none_count = check_none_target(train_loader)
val_none_count = check_none_target(val_loader)
test_none_count = check_none_target(test_loader)

print(f"Numero di target con None nel train dataset: {train_none_count}")
print(f"Numero di target con None nel validation dataset: {val_none_count}")
print(f"Numero di target con None nel test dataset: {test_none_count}")


Numero di target con None nel train dataset: 0
Numero di target con None nel validation dataset: 0
Numero di target con None nel test dataset: 0


## Check DataLoader

In [112]:
# Numero totale di campioni per ogni DataLoader
train_size = len(train_loader.dataset)
val_size = len(val_loader.dataset)
test_size = len(test_loader.dataset)

# Numero di batch per ogni DataLoader
train_batches = len(train_loader)
val_batches = len(val_loader)
test_batches = len(test_loader)

# Visualizza i risultati
print(f"Numero totale di elementi nel train_loader: {train_size}")
print(f"Numero totale di batch nel train_loader: {train_batches}")
print(f"Numero totale di elementi nel val_loader: {val_size}")
print(f"Numero totale di batch nel val_loader: {val_batches}")
print(f"Numero totale di elementi nel test_loader: {test_size}")
print(f"Numero totale di batch nel test_loader: {test_batches}")

# Somma totale degli elementi nei DataLoader
total_elements = train_size + val_size + test_size
print(f"Numero totale di elementi in tutti i DataLoader: {total_elements}")

Numero totale di elementi nel train_loader: 42172
Numero totale di batch nel train_loader: 21086
Numero totale di elementi nel val_loader: 21054
Numero totale di batch nel val_loader: 10527
Numero totale di elementi nel test_loader: 21177
Numero totale di batch nel test_loader: 10589
Numero totale di elementi in tutti i DataLoader: 84403


# Modello Faster R-CNN (Resnet50)

In [113]:
def compute_class_weights(dataset):
    # Conta la frequenza di ogni classe nel dataset
    class_counts = np.zeros(len(dataset.classes))  # Non consideriamo lo sfondo, quindi senza +1
    
    # Usa tqdm per monitorare il progresso mentre si itera sul dataset
    for _, targets in tqdm(dataset, desc="Calcolo frequenze delle classi", leave=False):
        # Controlla se targets è None
        if targets is None:
            continue
        
        # Assicurati che 'labels' sia un array e itera su di esso
        if 'labels' in targets:
            for target in targets['labels']:  
                class_counts[target] += 1

    # Calcola i pesi per le classi (senza lo sfondo)
    total_count = sum(class_counts)  

    # Calcola i pesi inversamente proporzionali alla frequenza
    class_weights = np.divide(total_count, class_counts)

    return class_weights

In [114]:
def train_and_validate(model, train_loader, val_loader, optimizer, device, class_weights, num_epochs=10, save_model=True, num_classes=12):
    """
    Funzione per il training e la validazione del modello Faster R-CNN con pesi delle classi e metriche di mAP e accuracy.
    """
    model.to(device)
    losses_per_epoch = []
    train_accuracies = []
    val_accuracies = []
    all_train_preds = defaultdict(list)
    all_train_labels = defaultdict(list)
    all_train_scores = defaultdict(list)
    all_val_preds = defaultdict(list)
    all_val_labels = defaultdict(list)
    all_val_scores = defaultdict(list)

    # Assicurati che i pesi siano un tensor PyTorch
    class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

    for epoch in range(num_epochs):
        print(f"\nEpoca {epoch + 1}/{num_epochs}")
        model.train()
        total_loss = 0

        # Training loop con tqdm
        train_loop = tqdm(train_loader, desc="Training", leave=False)
        for images, targets in train_loop:
            # Se targets è None (immagini senza annotazioni), crea un target con la classe "sfondo"
            if targets is None:
                targets = [{'boxes': torch.zeros((1, 4), device=device), 'labels': torch.tensor([0], device=device)} for _ in range(len(images))]
            else:
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            images = [img.to(device) for img in images]

            # Calcola le perdite
            loss_dict = model(images, targets)
            
            # Somma le perdite principali (classificazione, regressione delle box, etc.)
            losses = sum(loss for loss in loss_dict.values())

            # Verifica se le perdite sono per classe e ponderale
            for key in loss_dict:
                if 'loss' in key and loss_dict[key].dim() == 1:  # Se è per classe
                    losses += (loss_dict[key] * class_weights).sum()
                else:
                    losses += loss_dict[key]  # Per perdite scalari

            # Ottieni le predizioni per il calcolo delle metriche
            pred_labels = [t['labels'] for t in targets]
            pred_scores = [output['scores'] for output in model(images)]

            # Raccolta delle etichette predette e vere per la metrica
            for i, target in enumerate(targets):
                all_train_preds[i].extend(pred_labels[i].cpu().numpy())
                all_train_labels[i].extend([t.item() for t in target['labels']])
                all_train_scores[i].extend([score.item() for score in pred_scores[i]])

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

            total_loss += losses.item()
            train_loop.set_postfix(loss=losses.item())

        # Calcola l'accuratezza del training
        train_accuracy = accuracy_score(all_train_labels, all_train_preds)
        train_accuracies.append(train_accuracy)
        print(f"Perdita Totale per l'epoca {epoch + 1}: {total_loss:.4f} | Accuracy (Training): {train_accuracy:.4f}")

        losses_per_epoch.append(total_loss)

        # Validazione
        model.eval()
        val_loop = tqdm(val_loader, desc="Validazione", leave=False)

        with torch.no_grad():
            for images, targets in val_loop:
                # Se targets è None (immagini senza annotazioni), crea un target con la classe "sfondo"
                if targets is None:
                    targets = [{'boxes': torch.zeros((1, 4), device=device), 'labels': torch.tensor([0], device=device)} for _ in range(len(images))]
                else:
                    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                images = [img.to(device) for img in images]
                predictions = model(images)

                # Ottieni le predizioni per il calcolo delle metriche
                pred_labels = [t['labels'] for t in targets]
                pred_scores = [output['scores'] for output in predictions]

                # Raccolta delle etichette predette e vere per la metrica
                for i, target in enumerate(targets):
                    all_val_preds[i].extend(pred_labels[i].cpu().numpy())
                    all_val_labels[i].extend([t.item() for t in target['labels']])
                    all_val_scores[i].extend([score.item() for score in pred_scores[i]])

                val_loop.set_postfix(processed=len(predictions))  # Placeholder per metriche future

        # Calcola mAP
        ap = calculate_average_precision(all_val_scores, all_val_preds, all_val_labels, num_classes)
        mAP = sum(ap.values()) / num_classes

        # Calcola l'accuratezza della validazione
        val_accuracy = accuracy_score(all_val_labels, all_val_preds)
        val_accuracies.append(val_accuracy)

        print(f"Accuracy (Validation): {val_accuracy:.4f}")
        print(f"Mean Average Precision (mAP): {mAP:.4f}")

        # Salva il modello
        if save_model:
            torch.save(model.state_dict(), f"model_epoch_{epoch + 1}.pth")
            print(f"Modello salvato: model_epoch_{epoch + 1}.pth")

    # Plotting delle metriche
    epochs_range = range(num_epochs)

    plt.figure(figsize=(12, 8))

    # Loss plot
    plt.subplot(3, 1, 1)
    plt.plot(epochs_range, losses_per_epoch, label='Training Loss', color='blue')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training Loss per Epoca')

    # Accuracy plot
    plt.subplot(3, 1, 2)
    plt.plot(epochs_range, train_accuracies, label='Accuracy (Training)', color='green')
    plt.plot(epochs_range, val_accuracies, label='Accuracy (Validation)', color='orange')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Accuracy per Epoca')

    # mAP plot
    plt.subplot(3, 1, 3)
    plt.plot(epochs_range, [mAP] * num_epochs, label='mAP', color='red')
    plt.xlabel('Epochs')
    plt.ylabel('mAP')
    plt.legend()
    plt.title('mAP per Epoca')

    plt.tight_layout()
    plt.show()

    return losses_per_epoch, train_accuracies, val_accuracies, mAP

In [115]:
def test_model(model, test_loader, device):
    """
    Funzione per il testing del modello Faster R-CNN.
    
    Args:
    - model: il modello Faster R-CNN.
    - test_loader: DataLoader per il test set.
    - device: dispositivo su cui eseguire (es. 'cuda' o 'cpu').
    
    Returns:
    - predictions: lista delle predizioni per ogni batch (include 'boxes', 'labels', 'scores').
    """
    model.to(device)
    model.eval()
    predictions = []
    
    print("\nInizio testing...")
    test_loop = tqdm(test_loader, desc="Testing", leave=False)
    
    with torch.no_grad():
        for images, _ in test_loop:  # Durante il test, i target possono essere ignorati
            images = [img.to(device) for img in images]
            preds = model(images)
            
            # Predizioni di ciascun batch (contenente 'boxes', 'labels', 'scores')
            # Le predizioni sono in un formato di lista di dizionari
            for pred in preds:
                predictions.append({
                    'boxes': pred['boxes'].cpu().numpy(),
                    'labels': pred['labels'].cpu().numpy(),
                    'scores': pred['scores'].cpu().numpy()
                })
            
            # Aggiungi aggiornamenti su quante predizioni sono state processate
            test_loop.set_postfix(processed=len(predictions))

    print("Testing completato.")
    return predictions

In [116]:
# Carica il modello Faster R-CNN con ResNet50 e FPN
model = fasterrcnn_resnet50_fpn(weights=None)

num_classes = 13

# Modifica il numero di classi in output
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Imposta il dispositivo (GPU o CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Configurazione training
num_epochs = 2
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

In [117]:
# Calcola i pesi delle classi
class_weights = compute_class_weights(train_loader.dataset)

print(class_weights)

[5.42913361e+01 3.01860691e+00 1.97388871e+01 1.61693762e+02
 1.09486080e+02 1.26908012e+02 1.77176162e+00 4.62356757e+03
 3.39092170e+02 1.21155807e+02 1.62925714e+03            inf]


/tmp/ipykernel_23/1940286729.py:20: RuntimeWarning: divide by zero encountered in divide
  class_weights = np.divide(total_count, class_counts)


In [118]:
# Esegui il training
losses_per_epoch = train_and_validate(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device=device,
    class_weights=class_weights,
    num_epochs=num_epochs,
    save_model=True
)


Epoca 1/2


AssertionError: targets should not be none when in training mode

In [ ]:
# Chiamata alla funzione di test
predictions = test_model(model=model, test_loader=test_loader, device=device)

# Puoi fare qualcosa con le predizioni, come visualizzarle o calcolare metriche
print(f"Numero di predizioni ottenute: {len(predictions)}")